In [1]:
%reload_ext autoreload
%autoreload 2
from trainer import Trainer, TrainerConfig
from mingpt_utils import set_seed
from model import GPT, GPTConfig
import torch
from utils import *

from torch.utils.tensorboard import SummaryWriter
from mingpt_utils import sample
torch.cuda.empty_cache()

In [2]:
print("Available devices: ", torch.cuda.device_count())
print("torch version:", torch.__version__)
print("cudnn version:", torch.backends.cudnn.version())
print("cuda version:", torch.version.cuda)

Available devices:  2
torch version: 2.2.1+cu121
cudnn version: 8902
cuda version: 12.1


In [3]:

max_length = 1024
id = 0
tokens = np.load('../data/formatted/tokens.npy', allow_pickle=True)
train = np.load('../data/shuffled/dataset_train.npy', allow_pickle=True)
test = np.load('../data/shuffled/dataset_test.npy', allow_pickle=True)
midi_train = np.load('../data/shuffled/midi_train.npy', allow_pickle=True)
midi_test = np.load('../data/shuffled/midi_test.npy', allow_pickle=True)

In [ ]:
#Convert midi into dtype int
midi_train = midi_train.astype(int)
midi_test = midi_test.astype(int)

In [ ]:
print(train.shape, test.shape, midi_train.shape, midi_test.shape)

dataset = TokenDatasetMidi(train, midi_train,  max_length, tokens)
validation = TokenDatasetMidi(test, midi_test, max_length, tokens)

(43272, 2048) (4800, 2048) (43272, 2048, 8) (4800, 2048, 8)
data has 43272 pieces, 195 unique tokens.
data has 4800 pieces, 195 unique tokens.


In [ ]:
import wandb
#wandb.login()
wandb.init(
    # set the wandb project where this run will be logged
    project="music_gpt_new_voicing",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 3e-5,
    "architecture": "Transformer - minGPT",
    "dataset": "chords from iRealPro",
    "epochs": 90,
    }
)

03/14/2024 15:55:20 - ERROR - wandb.jupyter -   Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dazzid (music_gpt). Use `wandb login --relogin` to force relogin


In [ ]:
# import subprocess

# # Example command to list processes using GPU (this won't run here due to sandbox restrictions)
# output = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
# print(output.stdout)

Thu Mar 14 15:55:23 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         On | 00000000:1A:00.0 Off |                  N/A |
| 30%   26C    P8               22W / 350W|      3MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
epochs = 120
embedding = 256
heads = 6
layers = 6
batch_size = 32
learning_rate = 3e-5
num_workers = 6
midi_vocab = 128

mconf = GPTConfig(len(tokens), dataset.block_size, midi_vocab, n_layer=layers, n_head=heads, n_embd=embedding)
session_model = GPT(mconf)
MODEL_NAME = "../models/model_"+ "epochs->" + str(epochs) + "_heads->" + str(heads) + "_embd->" + str(embedding) + "_batch->" + str(batch_size) + "_new_midi_embeddings"
print(MODEL_NAME)

session_model = load_model(MODEL_NAME, session_model)

if (session_model == None):
    #mconf = GPTConfig(len(tokens), dataset.block_size, n_layer=layers, n_head=heads, n_embd=embbedings)
    session_model = GPT(mconf)
    tconf = TrainerConfig(max_epochs=epochs, 
                          batch_size=batch_size, 
                          learning_rate=learning_rate, 
                          num_workers=num_workers
                          )
    writer = SummaryWriter(log_dir='../runs/'+'logs') 
    trainer = Trainer(session_model, dataset, validation, tconf, writer)
    trainer.train()
    save_model(MODEL_NAME, session_model)
    # [optional] finish the wandb run, necessary in notebooks
    wandb.finish()

03/14/2024 15:55:23 - INFO - model -   number of parameters: 3.268096e+06
03/14/2024 15:55:23 - INFO - model -   number of parameters: 3.268096e+06


../models/model_epochs->90_heads->4_embd->256_batch->32_new_midi_embeddings


epoch 1 iter 1352: train loss 1.56734. lr 3.000000e-05: 100%|██████████| 1353/1353 [05:36<00:00,  4.02it/s]
03/14/2024 16:01:01 - INFO - trainer -   epoch train loss: 2.440773


train loss: 2.4407725049580633


03/14/2024 16:01:15 - INFO - trainer -   test loss: 1.572655
epoch 2 iter 1352: train loss 1.42762. lr 3.000000e-05: 100%|██████████| 1353/1353 [05:34<00:00,  4.04it/s]
03/14/2024 16:06:51 - INFO - trainer -   epoch train loss: 1.533085


...

train loss: 0.15376009871936777


03/15/2024 00:39:43 - INFO - trainer -   test loss: 0.163076


Model ../models/model_epochs->90_heads->4_embd->256_batch->32_new_midi_embeddings saved


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Train loss,█▅▄▄▄▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation loss,█▇▆▆▅▅▅▅▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
hits@1,▁▂▂▃▃▃▃▄▄▅▅▆▇▇▇▇▇███████████████████████
hits@3,▁▃▅▅▅▆▆▆▆▆▇█████████████████████████████
hits@5,▁▂▅▆▆▆▆▇▇▇▇█████████████████████████████
Train loss,0.15376
Validation loss,0.16308
hits@1,0.95892
hits@3,0.99104
hits@5,0.99462
